In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import argparse
import random
import numpy as np

import os

import data_loader
import model



In [2]:
def init_params(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            init.kaiming_normal_(m.weight, mode='fan_in')
            if m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            init.constant_(m.weight, 1)
            init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            init.normal_(m.weight, std=1e-3)
            if m.bias is not None:
                init.constant_(m.bias, 0)

In [3]:
trainloader, testloader= data_loader.load_data()

In [4]:
def train(trainloader, net, criterion, optimizer, use_cuda=True):
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        batch_size = inputs.size(0)
        total += batch_size
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)

        if (batch_idx == 0):
            print(outputs[0])
            print(targets[0])
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*batch_size
        _, predicted = torch.max(outputs.data, 1)
        correct += predicted.eq(targets.data).cpu().sum().item()

    return train_loss/total, 100 - 100.*correct/total

In [5]:
def test(testloader, net, criterion, use_cuda=True):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(testloader):
        batch_size = inputs.size(0)
        total += batch_size

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()*batch_size
        _, predicted = torch.max(outputs.data, 1)
        correct += predicted.eq(targets.data).cpu().sum().item()

    return test_loss/total, 100 - 100.*correct/total

In [6]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [7]:
lr = 0.01
start_epoch = 1
epoches = 30
criterion = nn.CrossEntropyLoss()
save_folder = 'VGG9'

net = model.VGG('VGG9')

if (torch.cuda.is_available()):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True
    net.cuda()
    criterion = criterion.cuda()

init_params(net)


In [8]:
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=0.0005)

In [9]:
if not os.path.exists('model_history/' + save_folder):
    os.makedirs('model_history/' + save_folder)


In [10]:
train_loss, train_err = test(trainloader, net, criterion, True)
test_loss, test_err = test(testloader, net, criterion, True)
status = 'e: %d loss: %.5f train_err: %.3f test_top1: %.3f test_loss %.5f \n' % (0, train_loss, train_err, test_err, test_loss)
print(status)

state = {
    'acc': 100 - test_err,
    'epoch': 0,
    'state_dict': net.state_dict()
}
opt_state = {
            'optimizer': optimizer.state_dict()
}
torch.save(state, 'model_history/' + save_folder + '/model_0.t7')
torch.save(opt_state, 'model_history/' + save_folder + '/opt_state_0.t7')

C:\Users\55366\anaconda3\envs\ml\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


e: 0 loss: 2.30273 train_err: 93.913 test_top1: 93.590 test_loss 2.30272 



In [11]:
for epoch in range(start_epoch, 10 + 1):
    loss, train_err = train(trainloader, net, criterion, optimizer, True)
    test_loss, test_err = test(testloader, net, criterion, True)

    status = 'e: %d loss: %.5f train_err: %.3f test_top1: %.3f test_loss %.5f \n' % (epoch, loss, train_err, test_err, test_loss)
    print(status)

    # Save checkpoint.
    acc = 100 - test_err
    state = {
        'acc': acc,
        'epoch': epoch,
        'state_dict': net.state_dict(),
    }
    opt_state = {
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, 'model_history/' + save_folder + '/model_' + str(epoch) + '.t7')
    torch.save(opt_state, 'model_history/' + save_folder + '/opt_state_' + str(epoch) + '.t7')

tensor([ 0.0015, -0.0166,  0.0094,  0.0096,  0.0246,  0.0047,  0.0005,  0.0049,
        -0.0016,  0.0085], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(2, device='cuda:0')
e: 1 loss: 0.15448 train_err: 4.887 test_top1: 3.220 test_loss 0.10485 

tensor([-5.8457, -0.2898,  0.6307,  7.6109, -5.4577, -1.7254, -6.6332, -0.0481,
        -1.5152, -2.3648], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(3, device='cuda:0')
e: 2 loss: 0.08048 train_err: 2.523 test_top1: 4.190 test_loss 0.14281 

tensor([-2.0809, -6.3510, -4.4985, -3.1898,  0.4309,  1.2129, -4.8235, -1.5452,
         0.6902,  7.2139], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(9, device='cuda:0')
e: 3 loss: 0.06879 train_err: 2.112 test_top1: 3.490 test_loss 0.10791 

tensor([-2.0809, -2.9806, 10.3352,  0.8104, -3.2804, -5.9496, -3.9364,  1.3120,
        -0.3134, -2.8726], device='cuda:0', grad_fn=<SelectBackward0>)
tensor(2, device='cuda:0')
e: 4 loss: 0.06302 train_err: 1.912 test_top1: 2.760 test_loss 0.0899